In [ ]:
# Import all necessary libraries and setup the environment for matplotlib
%matplotlib inline
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col
import numpy as np
import matplotlib.pyplot as plt
from numpy import *
import operator
import csv
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# $example off$
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit


In [ ]:
spark = SparkSession \
    .builder \
    .appName("Python Spark Machine Learning") \
    .getOrCreate()

    
train_datafile = "file:///home/stone/Train-label-28x28.csv"
test_datafile = "file:///home/stone/Test-label-28x28.csv"
    
#train_datafile = "file:///home/stone/Train-6000.csv"
#test_datafile = "file:///home/stone/Test-1000.csv"


train_df = spark.read.csv(train_datafile,inferSchema="true")
test_df = spark.read.csv(test_datafile,inferSchema="true")
#test_df.withColumnRenamed(, "label")
type(train_df)


In [ ]:
train_assembler = VectorAssembler(inputCols=train_df.columns[1:],
    outputCol="features")
train_vectors = train_assembler.transform(train_df).select(col(train_df.columns[0]).alias("labels"),"features")

test_assembler = VectorAssembler(inputCols=test_df.columns[1:],
    outputCol="features")
test_vectors = train_assembler.transform(train_df).select(col(train_df.columns[0]).alias("labels"),"features")


#train_vectors.show(5)
train_featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=10).fit(train_vectors)
train_labelIndexer = StringIndexer(inputCol="labels", outputCol="indexedLabel").fit(train_vectors)


In [ ]:
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=3)

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                                   labels=train_labelIndexer.labels)

In [ ]:
 pipeline = Pipeline(stages=[train_labelIndexer, train_featureIndexer, rf, labelConverter])

In [ ]:
 model = pipeline.fit(train_vectors)

In [ ]:
 predictions = model.transform(test_vectors)

In [ ]:
 predictions.select("predictedLabel", "labels", "features").show(4)

In [ ]:
evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

In [ ]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("accuracy = %g" % ( accuracy))

In [ ]:
 rfModel = model.stages[2]

In [ ]:
 print(rfModel)

In [ ]:
spark.stop()